In [ ]:
import pyspark
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)


In [ ]:
sc.stop()

In [ ]:
def load_vector(file: str):
    textFile = sc.textFile(file)
    def map_func(value):
        i, v = value.split()
        return int(i), float(v)
    return textFile.map(map_func)


def load_matrix(file: str):
    textFile = sc.textFile(file)
    def map_func(value):
        i, j, v = value.split()
        return int(i), int(j), float(v)
    return textFile.map(map_func)


V = load_vector("data/V.txt")
W = load_vector("data/W.txt")
M = load_matrix("data/M.txt")


In [ ]:
(
    V.collect(),
    W.collect(),
    M.collect()
)

In [ ]:
from operator import add
def map_func(value):
    _, v = value
    return v * v

normV = V.map(map_func).reduce(add) ** 0.5


In [ ]:
normV


In [ ]:
from operator import add
sumU = V.union(W).reduceByKey(add)
sumU.collect()

In [ ]:
from operator import add


def map_func(value):
    _, (vi, wi) = value
    return vi * wi

V.join(W).map(map_func).reduce(add)

In [ ]:
from operator import add

def map_func(value):
    i, j, v = value
    return j, (i, v)


sumVM = V.join(M.map(map_func))

def map_func(value):
    _, (v, (i, m)) = value
    return i, (m * v)


sumVM = sumVM.map(map_func)

sumVM = sumVM.reduceByKey(add)


In [ ]:
sumVM.collect()

In [ ]:
from operator import add


def norm(V) -> float:
    def map_func(value):
        _, v = value
        return v * v
    return V.map(map_func).reduce(add) ** 0.5

def matrix_mult(M, V):
    def map_func(value):
        i, j, v = value
        return j, (i, v)
    sumVM = V.join(M.map(map_func))
    def map_func(value):
        j, (v, (i, m)) = value
        return i, (m * v)
    sumVM = sumVM.map(map_func)
    return sumVM.reduceByKey(add)

def dot_product(V, W):
    def map_func(value):
        _, (vi, wi) = value
        return vi * wi
    return V.join(W).map(map_func).reduce(add)

Vi = V

for i in range(10):
    normV = norm(Vi)
    def map_func(value):
        i, v = value
        return i, v / normV
    NVi = Vi.map(map_func)
    Vi = matrix_mult(M, NVi)
    l = dot_product(Vi, NVi)
    print("For i =", i,"  l =", l)
    
